In [2]:
from pathlib import Path
import sys
sys.path.insert(1, '..')

from Sen2Flood import Sen2Flood
from plots import plot_curves
from unet import UNet
import torch
import torch.nn as nn
import numpy
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print('Device:', DEVICE)

Device: cuda


In [4]:
# load model
model_path = '/home/bruno/dataset_Sen2Flood/models/UNet_SouthAsia.pt'
model = UNet(in_channels=5, out_channels=2, dropout_val=0.2)
model.load_state_dict(torch.load(model_path, weights_only=True))
model = model.to(DEVICE)
model.eval()

UNet(
  (down_conv1): DownSample(
    (conv): DoubleConv(
      (conv_op): Sequential(
        (0): Sequential(
          (0): Conv2d(5, 64, kernel_size=(3, 3), stride=(1, 1))
          (1): Dropout(p=0.2, inplace=False)
          (2): LeakyReLU(negative_slope=0.01, inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
        )
        (1): Dropout(p=0.2, inplace=False)
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv2): DownSample(
    (conv): DoubleConv(
      (conv_op): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
              (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (1): Dropout(p=0.2, inplace=False)
            (2): LeakyReLU(negative_slope=0.0

In [6]:
from torchviz import make_dot

yhat = torch.zeros(32,5,256,256).to(DEVICE)

make_dot(model(yhat), params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

'rnn_torchviz.png'

In [3]:
dataset = Sen2Flood(dataset_folder='/media/bruno/Matosak/Sen2Flood', chip_size=256,
                        data_to_include=['s1_during_flood', 'terrain', 'flood_mask'],
                        percentile_scale_bttm=5, percentile_scale_top=95,
                        countries=['Bangladesh', 'India', 'Pakistan', 'Sri Lanka', 'Afghanistan', 'Nepal', 'Buthan'],
                        use_data_augmentation=True)

ValueError: "/media/bruno/Matosak/Sen2Flood" is not a folder.